## compute tSNR for each subject, protocol and session

In [1]:
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
from nipype.algorithms.confounds import TSNR as tsnr 
import nipype.interfaces.fsl.maths as fsl
import pandas as pd
import os, re, json

210702-19:21:56,198 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
tsnrwf = pe.Workflow(name = "tsnrwf")

## grab and save the data

In [3]:
# grab data
get_T2s = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                              outfields=['func', 'mask']), name = 'get_T2s')
get_T2s.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
get_T2s.inputs.sort_filelist = True
get_T2s.inputs.template = '*'
get_T2s.inputs.template_args = {'func': [['sub', 'sub', 'TR']],
                                'mask': [['sub', 'sub', 'TR']]}
get_T2s.inputs.field_template = {'func': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-*/func/sub-%s_*TR%s_space-T1w_desc-preproc_bold.nii.gz',
                                 'mask': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-*/func/sub-%s_*TR%s_space-T1w_desc-brain_mask.nii.gz'}

In [4]:
# test it
get_T2s.inputs.sub = '04'
get_T2s.inputs.TR = '700'
res = get_T2s.run()
res.outputs

210702-19:21:56,890 nipype.workflow INFO:
	 [Node] Setting-up "get_T2s" in "/tmp/tmpnumx0wgk/get_T2s".
210702-19:21:56,896 nipype.workflow INFO:
	 [Node] Running "get_T2s" ("nipype.interfaces.io.DataGrabber")
210702-19:21:56,906 nipype.workflow INFO:
	 [Node] Finished "get_T2s".



func = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-02/func/sub-04_ses-02_task-learnAtt_acq-TR700_space-T1w_desc-preproc_bold.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-03/func/sub-04_ses-03_task-learnAtt_acq-TR700_space-T1w_desc-preproc_bold.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-04/func/sub-04_ses-04_task-learnAtt_acq-TR700_space-T1w_desc-preproc_bold.nii.gz']
mask = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-02/func/sub-04_ses-02_task-learnAtt_acq-TR700_space-T1w_desc-brain_mask.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-03/func/sub-04_ses-03_task-learnAtt_acq-TR700_space-T1w_desc-brain_mask.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-04/func/sub-04_ses-04_task-learnAtt_acq-TR700_space-T1w_desc-brain_mask.nii.gz']

In [5]:
# feed data into the grabber
T2inf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                name='T2-info')
T2inf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
T2inf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

In [6]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    l = fname.split(os.sep)
    TR = re.search('.*acq-TR([0-9]*)_.*',l[-1])
    TR = str(int(TR.group(1)))
    subfol = [s for s in l if re.search('sub', s)][0]
#    sesfol = [s for s in l if re.search('ses', s)][0]
    name = [subfol, "TR"+TR]
    name
    name = '/'.join(name)
    return name

In [7]:
# test it
printSubPath(res.outputs.func)

'sub-04/TR700'

In [8]:
# sink it
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_do-msk([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

# apply brain mask

In [9]:
domask = pe.MapNode(fsl.ApplyMask(), iterfield = ['in_file', 'mask_file'], name = 'do-msk')

# tSNR operation

In [10]:
dotsnr = pe.MapNode(tsnr(), iterfield = ['in_file'], name = 'do-tsnr')

# connect workflow

In [11]:
tsnrwf.connect([(T2inf, get_T2s, [('sub',  'sub')]), 
             (T2inf, get_T2s, [('TR',   'TR')]),
             (get_T2s, ds, [(('func', printSubPath),
                              'container')]),
             (get_T2s, domask, [('func', 'in_file')]),
             (get_T2s, domask, [('mask', 'mask_file')]),
             (domask, dotsnr, [('out_file', 'in_file')]),
             (dotsnr, ds, [('tsnr_file', 'SNR.@snr')])
            ])
tsnrwf.run()

210702-19:21:56,979 nipype.workflow INFO:
	 Workflow tsnrwf settings: ['check', 'execution', 'logging', 'monitoring']
210702-19:21:57,51 nipype.workflow INFO:
	 Running serially.
210702-19:21:57,52 nipype.workflow INFO:
	 [Node] Setting-up "tsnrwf.get_T2s" in "/tmp/tmpnumx0wgk/get_T2s".
210702-19:21:57,59 nipype.workflow INFO:
	 [Node] Running "get_T2s" ("nipype.interfaces.io.DataGrabber")
210702-19:21:57,66 nipype.workflow INFO:
	 [Node] Finished "tsnrwf.get_T2s".
210702-19:21:57,66 nipype.workflow INFO:
	 [Node] Setting-up "tsnrwf.do-msk" in "/tmp/tmpvhs9qyhb/tsnrwf/_TR_1920_sub_05/do-msk".
210702-19:21:57,74 nipype.workflow INFO:
	 [Node] Setting-up "_do-msk0" in "/tmp/tmpvhs9qyhb/tsnrwf/_TR_1920_sub_05/do-msk/mapflow/_do-msk0".
210702-19:21:57,78 nipype.workflow INFO:
	 [Node] Running "_do-msk0" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-05/ses-02/func/sub-05_ses-02_task-learnAtt_acq-TR1920

210702-19:28:41,803 nipype.workflow INFO:
	 [Node] Setting-up "_do-tsnr0" in "/tmp/tmp3gfkriks/tsnrwf/_TR_1510_sub_05/do-tsnr/mapflow/_do-tsnr0".
210702-19:28:41,809 nipype.workflow INFO:
	 [Node] Running "_do-tsnr0" ("nipype.algorithms.confounds.TSNR")
210702-19:29:06,506 nipype.workflow INFO:
	 [Node] Finished "_do-tsnr0".
210702-19:29:06,508 nipype.workflow INFO:
	 [Node] Setting-up "_do-tsnr1" in "/tmp/tmp3gfkriks/tsnrwf/_TR_1510_sub_05/do-tsnr/mapflow/_do-tsnr1".
210702-19:29:06,513 nipype.workflow INFO:
	 [Node] Running "_do-tsnr1" ("nipype.algorithms.confounds.TSNR")
210702-19:29:31,326 nipype.workflow INFO:
	 [Node] Finished "_do-tsnr1".
210702-19:29:31,328 nipype.workflow INFO:
	 [Node] Setting-up "_do-tsnr2" in "/tmp/tmp3gfkriks/tsnrwf/_TR_1510_sub_05/do-tsnr/mapflow/_do-tsnr2".
210702-19:29:31,333 nipype.workflow INFO:
	 [Node] Running "_do-tsnr2" ("nipype.algorithms.confounds.TSNR")
210702-19:29:56,566 nipype.workflow INFO:
	 [Node] Finished "_do-tsnr2".
210702-19:29:56,570

210702-19:33:56,509 nipype.workflow INFO:
	 [Node] Finished "_do-msk0".
210702-19:33:56,511 nipype.workflow INFO:
	 [Node] Setting-up "_do-msk1" in "/tmp/tmpjhpa8ujc/tsnrwf/_TR_1920_sub_04/do-msk/mapflow/_do-msk1".
210702-19:33:56,516 nipype.workflow INFO:
	 [Node] Running "_do-msk1" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-03/func/sub-04_ses-03_task-learnAtt_acq-TR1920_space-T1w_desc-preproc_bold.nii.gz -mas /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-04/ses-03/func/sub-04_ses-03_task-learnAtt_acq-TR1920_space-T1w_desc-brain_mask.nii.gz /tmp/tmpjhpa8ujc/tsnrwf/_TR_1920_sub_04/do-msk/mapflow/_do-msk1/sub-04_ses-03_task-learnAtt_acq-TR1920_space-T1w_desc-preproc_bold_masked.nii.gz
210702-19:34:47,13 nipype.workflow INFO:
	 [Node] Finished "_do-msk1".
210702-19:34:47,15 nipype.workflow INFO:
	 [Node] Setting-up "_do-msk2" in "/tmp/tmpjhpa8ujc/tsnrwf/_TR_1920_sub_04/do-msk/mapflow/_do-m

210702-19:41:22,689 nipype.workflow INFO:
	 [Node] Running "sink-stuff" ("nipype.interfaces.io.DataSink")
210702-19:41:22,691 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-04/TR1510/SNR/_TR_1510_sub_04/_do-tsnr0/tsnr.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-04/TR1510/SNR//_do-tsnr0/tsnr.nii.gz
210702-19:41:22,710 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-04/TR1510/SNR/_TR_1510_sub_04/_do-tsnr1/tsnr.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-04/TR1510/SNR//_do-tsnr1/tsnr.nii.gz
210702-19:41:22,727 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-04/TR1510/SNR/_TR_1510_sub_04/_do-tsnr2/tsnr.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-04/TR1510/SNR//_do-tsnr2/tsnr.nii.gz
210702-19:41:22,749 nipype.workflow INFO:
	 [Node] Finished "tsnrwf.sink-stuff".
210702-19:41:22,750 nipype.workflow INFO:
	 [Node] Setting-up "tsnrwf.get_T2s" in

210702-19:46:44,407 nipype.workflow INFO:
	 [Node] Setting-up "_do-tsnr0" in "/tmp/tmpgc8avnev/tsnrwf/_TR_1920_sub_03/do-tsnr/mapflow/_do-tsnr0".
210702-19:46:44,411 nipype.workflow INFO:
	 [Node] Running "_do-tsnr0" ("nipype.algorithms.confounds.TSNR")
210702-19:47:05,229 nipype.workflow INFO:
	 [Node] Finished "_do-tsnr0".
210702-19:47:05,231 nipype.workflow INFO:
	 [Node] Setting-up "_do-tsnr1" in "/tmp/tmpgc8avnev/tsnrwf/_TR_1920_sub_03/do-tsnr/mapflow/_do-tsnr1".
210702-19:47:05,235 nipype.workflow INFO:
	 [Node] Running "_do-tsnr1" ("nipype.algorithms.confounds.TSNR")
210702-19:47:24,943 nipype.workflow INFO:
	 [Node] Finished "_do-tsnr1".
210702-19:47:24,948 nipype.workflow INFO:
	 [Node] Finished "tsnrwf.do-tsnr".
210702-19:47:24,949 nipype.workflow INFO:
	 [Node] Setting-up "tsnrwf.sink-stuff" in "/tmp/tmpweazl1_6/tsnrwf/_TR_1920_sub_03/sink-stuff".
210702-19:47:24,957 nipype.workflow INFO:
	 [Node] Running "sink-stuff" ("nipype.interfaces.io.DataSink")
210702-19:47:24,958 nip

210702-19:54:13,478 nipype.workflow INFO:
	 [Node] Finished "_do-msk1".
210702-19:54:13,482 nipype.workflow INFO:
	 [Node] Setting-up "_do-msk2" in "/tmp/tmpchi2lq49/tsnrwf/_TR_700_sub_03/do-msk/mapflow/_do-msk2".
210702-19:54:13,487 nipype.workflow INFO:
	 [Node] Running "_do-msk2" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-03/ses-04/func/sub-03_ses-04_task-learnAtt_acq-TR700_space-T1w_desc-preproc_bold.nii.gz -mas /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-03/ses-04/func/sub-03_ses-04_task-learnAtt_acq-TR700_space-T1w_desc-brain_mask.nii.gz /tmp/tmpchi2lq49/tsnrwf/_TR_700_sub_03/do-msk/mapflow/_do-msk2/sub-03_ses-04_task-learnAtt_acq-TR700_space-T1w_desc-preproc_bold_masked.nii.gz
210702-19:55:10,409 nipype.workflow INFO:
	 [Node] Finished "_do-msk2".
210702-19:55:10,413 nipype.workflow INFO:
	 [Node] Finished "tsnrwf.do-msk".
210702-19:55:10,413 nipype.workflow INFO:
	 [Node] Setting-up "

210702-19:59:52,454 nipype.workflow INFO:
	 [Node] Finished "tsnrwf.sink-stuff".
210702-19:59:52,455 nipype.workflow INFO:
	 [Node] Setting-up "tsnrwf.get_T2s" in "/tmp/tmpnumx0wgk/get_T2s".
210702-19:59:52,460 nipype.workflow INFO:
	 [Node] Running "get_T2s" ("nipype.interfaces.io.DataGrabber")
210702-19:59:52,468 nipype.workflow INFO:
	 [Node] Finished "tsnrwf.get_T2s".
210702-19:59:52,469 nipype.workflow INFO:
	 [Node] Setting-up "tsnrwf.do-msk" in "/tmp/tmpmg4e7fcw/tsnrwf/_TR_1510_sub_02/do-msk".
210702-19:59:52,477 nipype.workflow INFO:
	 [Node] Setting-up "_do-msk0" in "/tmp/tmpmg4e7fcw/tsnrwf/_TR_1510_sub_02/do-msk/mapflow/_do-msk0".
210702-19:59:52,482 nipype.workflow INFO:
	 [Node] Running "_do-msk0" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-02/ses-02/func/sub-02_ses-02_task-learnAtt_acq-TR1510_space-T1w_desc-preproc_bold.nii.gz -mas /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-02/se

210702-20:07:58,564 nipype.workflow INFO:
	 [Node] Setting-up "_do-tsnr0" in "/tmp/tmpikhwb53y/tsnrwf/_TR_700_sub_02/do-tsnr/mapflow/_do-tsnr0".
210702-20:07:58,569 nipype.workflow INFO:
	 [Node] Running "_do-tsnr0" ("nipype.algorithms.confounds.TSNR")
210702-20:08:16,96 nipype.workflow INFO:
	 [Node] Finished "_do-tsnr0".
210702-20:08:16,99 nipype.workflow INFO:
	 [Node] Setting-up "_do-tsnr1" in "/tmp/tmpikhwb53y/tsnrwf/_TR_700_sub_02/do-tsnr/mapflow/_do-tsnr1".
210702-20:08:16,104 nipype.workflow INFO:
	 [Node] Running "_do-tsnr1" ("nipype.algorithms.confounds.TSNR")
210702-20:08:34,175 nipype.workflow INFO:
	 [Node] Finished "_do-tsnr1".
210702-20:08:34,178 nipype.workflow INFO:
	 [Node] Setting-up "_do-tsnr2" in "/tmp/tmpikhwb53y/tsnrwf/_TR_700_sub_02/do-tsnr/mapflow/_do-tsnr2".
210702-20:08:34,183 nipype.workflow INFO:
	 [Node] Running "_do-tsnr2" ("nipype.algorithms.confounds.TSNR")
210702-20:08:52,202 nipype.workflow INFO:
	 [Node] Finished "_do-tsnr2".
210702-20:08:52,207 nipy

210702-20:13:36,623 nipype.workflow INFO:
	 [Node] Running "_do-tsnr0" ("nipype.algorithms.confounds.TSNR")
210702-20:14:01,815 nipype.workflow INFO:
	 [Node] Finished "_do-tsnr0".
210702-20:14:01,819 nipype.workflow INFO:
	 [Node] Setting-up "_do-tsnr1" in "/tmp/tmp9eeh2r1f/tsnrwf/_TR_1510_sub_01/do-tsnr/mapflow/_do-tsnr1".
210702-20:14:01,823 nipype.workflow INFO:
	 [Node] Running "_do-tsnr1" ("nipype.algorithms.confounds.TSNR")
210702-20:14:26,314 nipype.workflow INFO:
	 [Node] Finished "_do-tsnr1".
210702-20:14:26,319 nipype.workflow INFO:
	 [Node] Finished "tsnrwf.do-tsnr".
210702-20:14:26,320 nipype.workflow INFO:
	 [Node] Setting-up "tsnrwf.sink-stuff" in "/tmp/tmprgy21rkh/tsnrwf/_TR_1510_sub_01/sink-stuff".
210702-20:14:26,328 nipype.workflow INFO:
	 [Node] Running "sink-stuff" ("nipype.interfaces.io.DataSink")
210702-20:14:26,330 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-01/TR1510/SNR/_TR_1510_sub_01/_do-tsnr0/tsnr.nii.gz -> /scratch/qbi/